In [10]:
from scipy.special import softmax
import torch
import pandas as pd
import numpy as np
import math

In [11]:
math.log(10000.0) 

9.210340371976184

In [2]:
softmax([14, 12])

array([0.88079708, 0.11920292])

In [4]:
data_dict = {
    'ETTh1':"Dataset_ETT_hour",
    'ETTh2':"Dataset_ETT_hour",
    'ETTm1':"Dataset_ETT_minute",
    'ETTm2':"Dataset_ETT_minute",
    'WTH':"Dataset_Custom",
    'ECL':"Dataset_Custom",
    'Solar':"Dataset_Custom",
    'custom':"Dataset_Custom",
}

In [20]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080'

In [18]:
torch.cuda.get_device_capability(0)

(6, 1)

In [19]:
!nvidia-smi

Mon Jan  9 14:43:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:1C:00.0  On |                  N/A |
|  0%   46C    P8    17W / 240W |   2067MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [50]:
df = pd.read_pickle('data/ETT/capacity_price_processed.pkl')
df['date'] = df['date'].apply(lambda cell: cell.strftime("%Y-%m-%d %H:%M:%S"))

product_types = df['product'].unique()

save_path = 'data/ETT/'

for prod in product_types:
    df_ = df[df['product'] == prod][['date', 'capacity_price']]
    save_name = f'SRL_{prod}.csv'
    df_.to_csv(f'{save_path}{save_name}', index=False)

In [11]:
df_ = df[df['product'] == 'NEG_00_04']

In [41]:
df_ = pd.read_csv('data/ETT/SRL_NEG_00_04.csv')

In [38]:
df['date'] = df['date'].apply(lambda cell: cell.strftime("%Y-%m-%d %H:%M:%S"))

In [35]:
d = df['date'].strftime("%Y-%m-%d %H:%M:%S")

AttributeError: 'Series' object has no attribute 'strftime'

In [33]:
d.strftime("%Y-%m-%d %H:%M:%S")

'2020-11-01 00:00:00'

### Test: `utils.timefeatures.time_features()`

In [52]:
from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

def time_features(dates, timeenc=1, freq='h'):
    """
    > `time_features` takes in a `dates` dataframe with a 'dates' column and extracts the date down to `freq` where freq can be any of the following if `timeenc` is 0: 
    > * m - [month]
    > * w - [month]
    > * d - [month, day, weekday]
    > * b - [month, day, weekday]
    > * h - [month, day, weekday, hour]
    > * t - [month, day, weekday, hour, *minute]
    > 
    > If `timeenc` is 1, a similar, but different list of `freq` values are supported (all encoded between [-0.5 and 0.5]): 
    > * Q - [month]
    > * M - [month]
    > * W - [Day of month, week of year]
    > * D - [Day of week, day of month, day of year]
    > * B - [Day of week, day of month, day of year]
    > * H - [Hour of day, day of week, day of month, day of year]
    > * T - [Minute of hour*, hour of day, day of week, day of month, day of year]
    > * S - [Second of minute, minute of hour, hour of day, day of week, day of month, day of year]

    *minute returns a number from 0-3 corresponding to the 15 minute period it falls into.
    """
    if timeenc==0:
        dates['month'] = dates.date.apply(lambda row:row.month,1)
        dates['day'] = dates.date.apply(lambda row:row.day,1)
        dates['weekday'] = dates.date.apply(lambda row:row.weekday(),1)
        dates['hour'] = dates.date.apply(lambda row:row.hour,1)
        dates['minute'] = dates.date.apply(lambda row:row.minute,1)
        dates['minute'] = dates.minute.map(lambda x:x//15)
        freq_map = {
            'y':[],'m':['month'],'w':['month'],'d':['month','day','weekday'],
            'b':['month','day','weekday'],'h':['month','day','weekday','hour'],
            't':['month','day','weekday','hour','minute'],
        }
        return dates[freq_map[freq.lower()]].values
    if timeenc==1:
        dates = pd.to_datetime(dates.date.values)
        return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)]).transpose(1,0)

In [53]:
df = pd.read_csv('data/ETT/SRL_NEG_00_04.csv')

In [69]:
dates

,date
0,2020-11-01 00:00:00
1,2020-11-02 00:00:00
2,2020-11-03 00:00:00
3,2020-11-04 00:00:00
4,2020-11-05 00:00:00
...,...
780,2022-12-21 00:00:00
781,2022-12-22 00:00:00
782,2022-12-23 00:00:00
783,2022-12-24 00:00:00


In [75]:
df_stamp = df[['date']]
df_stamp['date'] = pd.to_datetime(df_stamp.date)
stamp = time_features(df_stamp, timeenc=0, freq='d')

C:\Users\93cha\AppData\Local\Temp/ipykernel_17640/2697475164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stamp['date'] = pd.to_datetime(df_stamp.date)


In [81]:
stamp[0:10].shape

(10, 3)

### Test embedding codes

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1) # 2-dim vector max_len * 1
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

In [14]:
pe = PositionalEmbedding(64)
x = torch.randn(100)

In [236]:
pred.shape

(128, 24, 1)

In [241]:
test.__len__()

62

In [235]:
len(df_raw)*0.2

157.0

In [43]:
train[0][1]

array([[ 2.11210762],
       [ 2.13510142],
       [ 2.11973619],
       [ 2.24244099],
       [ 2.24244099],
       [ 2.23481283],
       [ 2.16583186],
       [ 2.33441492],
       [ 2.28079959],
       [ 2.16583186],
       [ 2.16583186],
       [ 1.38383375],
       [ 1.71348062],
       [ 2.07374902],
       [ 2.03549891],
       [ 2.25006914],
       [ 2.44175449],
       [ 2.30379338],
       [ 1.92042227],
       [ 1.81319162],
       [ 1.71348062],
       [ 1.74410215],
       [ 1.8744351 ],
       [ 1.98951174],
       [ 1.89742889],
       [ 1.69811498],
       [ 1.82844834],
       [ 1.91279412],
       [ 2.02013369],
       [ 1.88980074],
       [ 1.98177467],
       [ 1.82082019],
       [ 2.04312707],
       [ 2.00476847],
       [ 2.11973619],
       [ 2.20408197],
       [ 1.90516596],
       [ 1.82082019],
       [ 1.75183922],
       [ 1.91279412],
       [ 2.13510142],
       [ 1.92042227],
       [ 0.84724522],
       [ 1.71348062],
       [ 1.74410215],
       [ 1

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd

from exp.exp_informer import Exp_Informer

from utils.masking import TriangularCausalMask, ProbMask
from models.encoder import Encoder, EncoderLayer, ConvLayer, EncoderStack
from models.decoder import Decoder, DecoderLayer
from models.attn import FullAttention, ProbAttention, AttentionLayer
from models.embed import DataEmbedding

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from utils.tools import StandardScaler
from utils.timefeatures import time_features

from data.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred
from exp.exp_basic import Exp_Basic
from models.model import Informer, InformerStack

from utils.tools import EarlyStopping, adjust_learning_rate
from utils.metrics import metric

In [4]:
# Create dataset
df_raw = pd.read_csv('data/ETT/SRL_NEG_00_04.csv')

train =  Dataset_Custom('data/ETT/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price')
test =  Dataset_Custom('data/ETT/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price',flag='test')
vali =  Dataset_Custom('data/ETT/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price', flag='val')

In [5]:
# seq_x
len(train[0][0]) #seq_len = 384
# seq_y
len(train[0][1]) #label_len = 192


384

192

In [6]:
train_loader = DataLoader(train, batch_size=2)

In [7]:
sample_loader = next(iter(train_loader))
print(sample_loader[0])

tensor([[[ 8.2929e-02],
         [ 4.7766e-01],
         [ 1.7589e+00],
         [ 7.8655e-01],
         [ 1.1257e-01],
         [-9.2156e-02],
         [-1.8823e-01],
         [-2.2989e-01],
         [-2.0961e-01],
         [-2.2929e-01],
         [-2.6144e-01],
         [-2.7479e-01],
         [-2.8179e-01],
         [-2.9053e-01],
         [ 6.9259e-02],
         [ 3.3172e-01],
         [-2.6431e-02],
         [-1.9670e-01],
         [ 2.0738e-01],
         [-8.0400e-02],
         [-2.2601e-01],
         [ 6.9259e-02],
         [-7.2909e-02],
         [-1.5903e-01],
         [-1.7680e-01],
         [-1.8183e-01],
         [-2.1355e-01],
         [-2.4165e-01],
         [-2.3153e-01],
         [-2.3487e-01],
         [-2.1387e-01],
         [-2.4242e-01],
         [-2.5281e-01],
         [-2.2054e-01],
         [-2.6155e-01],
         [-2.8578e-01],
         [-2.7621e-01],
         [-2.5882e-01],
         [-2.5882e-01],
         [-2.6565e-01],
         [-2.4088e-01],
         [-2.369

In [8]:
batch_x, batch_y, batch_x_mark, batch_y_mark = sample_loader

In [112]:
batch_y.shape

torch.Size([2, 144, 1])

In [113]:
sample_loader[0].shape

torch.Size([2, 384, 1])

In [114]:
df_raw = pd.read_csv('data/ETT/ETTh1-mini.csv')

df_stamp = df_raw[['date']]
df_stamp['date'] = pd.to_datetime(df_stamp.date)
data_stamp = time_features(df_stamp, timeenc=0, freq='d')

In [115]:
df_scale = df_raw.iloc[:,1:]
scaler = StandardScaler()
scaler.fit(df_scale.values)

In [116]:
res = scaler.transform(df_scale.values)

In [126]:
[a for a in range(1,3)]

[1, 2]

In [128]:
class TestConv(nn.Module):
    def __init__(self, c_in, c_out):
        super(TestConv, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.conv = nn.Conv1d(in_channels=c_in, out_channels=c_out, kernel_size=3, padding=1,
                              padding_mode='circular')
        
    def forward(self, x):
        x = self.conv(x)
        return x

In [129]:
x = torch.Tensor(range(0, 15))

In [154]:
batch_x.shape

torch.Size([2, 384, 1])

In [146]:
d_emb = DataEmbedding(1, 512)

In [148]:
batch_x.shape

torch.Size([2, 384, 1])

### Plot prediction & true

In [11]:
pred = np.load('results/informer_SRL_NEG_00_04_ftM_sl96_ll48_pl24_dm64_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_test_1/pred.npy')

true = np.load('results/informer_SRL_NEG_00_04_ftM_sl96_ll48_pl24_dm64_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_test_1/true.npy')

In [10]:
pred.shape

(128, 24, 1)

In [12]:
pred = train.scaler.inverse_transform(pred)
true = train.scaler.inverse_transform(true)

In [13]:
df_raw[-25:-1]['capacity_price']

760    24.59
761    21.55
762    19.80
763    19.80
764    36.98
765    25.78
766    34.60
767    24.54
768    21.25
769    16.37
770    16.37
771    22.99
772    23.59
773    25.39
774    30.29
775    39.95
776    23.00
777    20.60
778    56.14
779    58.32
780    33.00
781    34.13
782    27.61
783    41.92
Name: capacity_price, dtype: float64

In [190]:
import matplotlib.pyplot as plt

In [216]:
np.array([[1, 2, 3], [4, 5, 6]])

array([[1, 2, 3],
       [4, 5, 6]])

In [214]:
np.flip(np.array([[1, 2, 3], [4, 5, 6]]), axis=-1)

array([[3, 2, 1],
       [6, 5, 4]])

In [217]:
pd.DataFrame(np.flip(.squeeze().transpose(), axis=-1))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999,27.650000,38.830000,49.000000,52.240000,78.120000
1,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999,27.650000,38.830000,49.000000,52.240000
2,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999,27.650000,38.830000,49.000000
3,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999,27.650000,38.830000
4,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999,27.650000
5,28.259999,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002,21.879999
6,24.590000,28.259999,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001,19.000002
7,21.549997,24.590000,28.259999,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000,29.730001
8,19.799997,21.549997,24.590000,28.259999,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999,22.800000
9,19.799997,19.799997,21.549997,24.590000,28.259999,33.649999,30.900001,29.150001,31.020001,43.330001,56.120000,42.199999,69.950000,27.540001,21.690000,27.730000,31.850000,45.030000,28.279999


In [233]:
np.array(df_raw[-134:-6]['capacity_price'])

array([ 5.35,  8.54,  5.82,  4.8 , 13.96,  8.92,  9.49,  9.  ,  8.69,
        7.  ,  6.12,  5.8 ,  5.06,  8.18, 15.48, 13.72, 10.15, 18.09,
       17.48, 11.59, 23.3 , 25.14, 68.32, 76.25, 55.66, 31.38, 24.48,
       20.14, 19.07, 17.92, 40.9 , 28.62, 21.56, 43.99, 46.91, 48.7 ,
       38.97, 33.85, 23.44, 18.52, 15.71, 14.61, 17.45, 29.34, 44.82,
       29.56, 23.8 , 18.  , 53.96, 81.2 , 75.21, 46.02, 49.6 , 69.06,
       45.9 , 43.45, 36.61, 43.33, 36.46, 30.  , 27.1 , 23.18, 24.  ,
       38.75, 32.5 , 24.81, 23.33, 45.57, 36.89, 28.11, 24.7 , 48.44,
       37.81, 27.7 , 21.66, 19.96, 16.77, 18.81, 16.47, 20.37, 34.56,
       29.  , 21.7 , 16.62, 27.45, 78.12, 52.24, 49.  , 38.83, 27.65,
       21.88, 19.  , 29.73, 22.8 , 28.28, 45.03, 31.85, 27.73, 21.69,
       27.54, 69.95, 42.2 , 56.12, 43.33, 31.02, 29.15, 30.9 , 33.65,
       28.26, 24.59, 21.55, 19.8 , 19.8 , 36.98, 25.78, 34.6 , 24.54,
       21.25, 16.37, 16.37, 22.99, 23.59, 25.39, 30.29, 39.95, 23.  ,
       20.6 , 56.14]

In [15]:
true.squeeze().transpose()[-1]

array([ 5.35000062,  8.54000183,  5.81999964,  4.80000154, 13.95999803,
        8.91999941,  9.49000124,  8.99999948,  8.69000059,  6.99999786,
        6.1200026 ,  5.80000235,  5.05999902,  8.18000154, 15.47999926,
       13.71999784, 10.14999904, 18.09000272, 17.48000087, 11.5900002 ,
       23.30000012, 25.13999888, 68.32      , 76.24999992, 55.66000034,
       31.38000119, 24.48000107, 20.14000029, 19.07000079, 17.92000122,
       40.90000069, 28.61999896, 21.55999871, 43.9899991 , 46.91000009,
       48.70000017, 38.9700005 , 33.84999909, 23.44000023, 18.52000171,
       15.70999808, 14.60999992, 17.45000221, 29.33999954, 44.81999977,
       29.55999972, 23.80000052, 18.00000129, 53.96000033, 81.20000051,
       75.20999977, 46.01999937, 49.59999954, 69.06000008, 45.90000064,
       43.44999866, 36.61000132, 43.33000129, 36.45999984, 30.00000007,
       27.10000046, 23.18000002, 24.00000068, 38.75000032, 32.49999936,
       24.80999997, 23.32999878, 45.57000037, 36.88999882, 28.10